In [ ]:
import os
import subprocess

# Required packages
required_packages = [
    "torch", "torchvision", "torch_geometric", "torchvision.models",
    "numpy", "scikit-learn"
]

# Install missing packages
for package in required_packages:
    try:
        __import__(package.split('.')[0])
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call(["pip", "install", package])

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import kneighbors_graph
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
import torch.nn.functional as F

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Auto-download Oxford Flowers 102 dataset
data_dir = './oxford_flowers102'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    # Convert grayscale images to 3 channels before normalization
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = datasets.Flowers102(root=data_dir, split='train', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

print(f"Dataset loaded: {len(dataset)} images.")

# Fine-tune ResNet101
resnet = models.resnet101(pretrained=True)
# Access the number of classes using the '_labels' attribute and a set to get unique classes
num_classes = len(set(dataset._labels)) # Changed 'targets' to '_labels'
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes) # Change this line
resnet.to(device)
optimizer = optim.AdamW(resnet.parameters(), lr=0.0003, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train ResNet101
def train_resnet(model, dataloader, optimizer, criterion, epochs=15):
    model.train()
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(epochs):
        total_loss, correct = 0, 0
        for img, label in dataloader:
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(img)
                loss = criterion(output, label)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            correct += (output.argmax(dim=1) == label).sum().item()
        accuracy = correct / len(dataset)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {accuracy:.4f}")
    print(f"Final ResNet Accuracy: {accuracy:.4f}")

train_resnet(resnet, dataloader, optimizer, criterion, epochs=15)

# Feature Extraction
resnet.fc = nn.Identity()
def extract_features(dataset, model):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for img, label in dataset:
            img = img.unsqueeze(0).to(device)
            feat = model(img).cpu().numpy().flatten()
            features.append(feat)
            labels.append(label)
    return np.array(features), np.array(labels)

features, labels = extract_features(dataset, resnet)
labels = LabelEncoder().fit_transform(labels)

# Build Optimized KNN Graph
def build_knn_graph(features, k=10):
    adj_matrix = kneighbors_graph(features, k, mode='connectivity', include_self=True).toarray()
    edge_index = np.array(np.nonzero(adj_matrix))
    return torch.tensor(edge_index, dtype=torch.long)

edge_index = build_knn_graph(features)

graph_data = Data(
    x=torch.tensor(features, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(labels, dtype=torch.long)
)

# Define Improved GCN Model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = BatchNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = BatchNorm(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.norm1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.norm2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize & Train GCN
gcn = GCN(in_channels=features.shape[1], hidden_channels=512, out_channels=len(set(labels))).to(device)
optimizer = optim.AdamW(gcn.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
loss_fn = nn.CrossEntropyLoss()

def train_gcn(model, data, optimizer, loss_fn, scheduler, epochs=100):
    model.train()
    best_acc = 0.0
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = loss_fn(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        acc = (out.argmax(dim=1) == data.y.to(device)).float().mean().item()
        best_acc = max(best_acc, acc)
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")
    print(f"Final GCN Accuracy: {best_acc:.4f}")

train_gcn(gcn, graph_data, optimizer, loss_fn, scheduler, epochs=100)

print("Training complete. Best accuracy achieved during training is displayed above.")

Installing scikit-learn...
Using device: cuda
Dataset loaded: 1020 images.


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResN

Epoch 1, Loss: 3.6855, Accuracy: 0.2627
Epoch 2, Loss: 1.5387, Accuracy: 0.6843
Epoch 3, Loss: 0.6482, Accuracy: 0.8784
Epoch 4, Loss: 0.3518, Accuracy: 0.9382
Epoch 5, Loss: 0.2850, Accuracy: 0.9392
Epoch 6, Loss: 0.2497, Accuracy: 0.9441
Epoch 7, Loss: 0.2070, Accuracy: 0.9578
Epoch 8, Loss: 0.1668, Accuracy: 0.9608
Epoch 9, Loss: 0.1453, Accuracy: 0.9686
Epoch 10, Loss: 0.0921, Accuracy: 0.9863
Epoch 11, Loss: 0.0536, Accuracy: 0.9892
Epoch 12, Loss: 0.0552, Accuracy: 0.9931
Epoch 13, Loss: 0.1216, Accuracy: 0.9716
Epoch 14, Loss: 0.1822, Accuracy: 0.9549
Epoch 15, Loss: 0.2030, Accuracy: 0.9569
Final ResNet Accuracy: 0.9569
Epoch 1, Loss: 5.2347, Accuracy: 0.0088
Epoch 2, Loss: 1.8476, Accuracy: 0.7294
Epoch 3, Loss: 0.6929, Accuracy: 0.9608
Epoch 4, Loss: 0.2887, Accuracy: 0.9775
Epoch 5, Loss: 0.1496, Accuracy: 0.9863
Epoch 6, Loss: 0.1048, Accuracy: 0.9922
Epoch 7, Loss: 0.0772, Accuracy: 0.9922
Epoch 8, Loss: 0.0567, Accuracy: 0.9980
Epoch 9, Loss: 0.0428, Accuracy: 0.9961
Epoc

In [ ]:
# Install required libraries
!pip install torch torchvision torch_geometric timm numpy scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm  # Using Vision Transformer (ViT)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import kneighbors_graph
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
import torch.nn.functional as F

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Advanced Data Augmentation
data_dir = './oxford_flowers102'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=30, shear=10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),  # Grayscale Fix
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Use zero-centered norm
])

# Load dataset
dataset = datasets.Flowers102(root=data_dir, split='train', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

print(f"Dataset loaded: {len(dataset)} images.")

# Use Vision Transformer (ViT) Instead of ResNet
vit = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=len(set(dataset._labels)))
vit.to(device)
optimizer = optim.AdamW(vit.parameters(), lr=0.0001, weight_decay=5e-5)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # ✅ Label Smoothing

# Train ViT for Feature Extraction
def train_vit(model, dataloader, optimizer, criterion, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss, correct = 0, 0
        for img, label in dataloader:
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            correct += (output.argmax(dim=1) == label).sum().item()
        accuracy = correct / len(dataset)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {accuracy:.4f}")
    print(f"Final ViT Accuracy: {accuracy:.4f}")

train_vit(vit, dataloader, optimizer, criterion, epochs=20)

# Feature Extraction
vit.head = nn.Identity()
def extract_features(dataset, model):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for img, label in dataset:
            img = img.unsqueeze(0).to(device)
            feat = model(img).cpu().numpy().flatten()
            features.append(feat)
            labels.append(label)
    return np.array(features), np.array(labels)

features, labels = extract_features(dataset, vit)
labels = LabelEncoder().fit_transform(labels)

# Higher Quality KNN Graph
def build_knn_graph(features, k=20):  # Increased k for better graph structure
    adj_matrix = kneighbors_graph(features, k, mode='distance', include_self=True).toarray()
    edge_index = np.array(np.nonzero(adj_matrix))
    return torch.tensor(edge_index, dtype=torch.long)

edge_index = build_knn_graph(features)

graph_data = Data(
    x=torch.tensor(features, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(labels, dtype=torch.long)
)

# Improved GCN Model (Deeper Architecture)
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = BatchNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = BatchNorm(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.norm3 = BatchNorm(hidden_channels)
        self.conv4 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.norm1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.norm2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index).relu()
        x = self.norm3(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)

# Train GCN with Optimized Hyperparameters
gcn = GCN(in_channels=features.shape[1], hidden_channels=1024, out_channels=len(set(labels))).to(device)
optimizer = optim.AdamW(gcn.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=75)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

def train_gcn(model, data, optimizer, loss_fn, scheduler, epochs=150):
    model.train()
    best_acc = 0.0
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = loss_fn(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        acc = (out.argmax(dim=1) == data.y.to(device)).float().mean().item()
        best_acc = max(best_acc, acc)
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")
    print(f"Final GCN Accuracy: {best_acc:.4f}")

train_gcn(gcn, graph_data, optimizer, loss_fn, scheduler, epochs=150)

print("Training complete. Best accuracy achieved during training is displayed above.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


100%|██████████| 345M/345M [00:13<00:00, 25.2MB/s]
100%|██████████| 502/502 [00:00<00:00, 962kB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 19.5MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Dataset loaded: 1020 images.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1, Loss: 4.4358, Accuracy: 0.0794
Epoch 2, Loss: 1.9928, Accuracy: 0.6706
Epoch 3, Loss: 1.1443, Accuracy: 0.9422
Epoch 4, Loss: 0.9735, Accuracy: 0.9873
Epoch 5, Loss: 0.9076, Accuracy: 0.9971
Epoch 6, Loss: 0.8818, Accuracy: 0.9941
Epoch 7, Loss: 0.8525, Accuracy: 0.9990
Epoch 8, Loss: 0.8373, Accuracy: 1.0000
Epoch 9, Loss: 0.8280, Accuracy: 1.0000
Epoch 10, Loss: 0.8228, Accuracy: 1.0000
Epoch 11, Loss: 0.8188, Accuracy: 1.0000
Epoch 12, Loss: 0.8152, Accuracy: 1.0000
Epoch 13, Loss: 0.8110, Accuracy: 1.0000
Epoch 14, Loss: 0.8076, Accuracy: 1.0000
Epoch 15, Loss: 0.8057, Accuracy: 1.0000
Epoch 16, Loss: 0.8037, Accuracy: 1.0000
Epoch 17, Loss: 0.8028, Accuracy: 1.0000
Epoch 18, Loss: 0.8012, Accuracy: 1.0000
Epoch 19, Loss: 0.8002, Accuracy: 1.0000
Epoch 20, Loss: 0.7987, Accuracy: 1.0000
Final ViT Accuracy: 1.0000
